In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

2024-03-15 17:31:42.205851: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 17:31:45.006512: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 17:31:45.006589: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 17:31:45.410094: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-15 17:31:46.169201: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 17:31:46.170291: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# Loading the data

# Getting the current working directory
current_dir = os.getcwd()

# Appending the data/mnist.npz to the previous path to get the full path
data_path = os.path.join(current_dir, "/workspaces/Portfolio/Machine Learning/C1/W3/mnist.npz")
# Discarding the test set
(training_images, training_labels) , _  = tf.keras.datasets.mnist.load_data(path=data_path)

In [3]:
# Function: reshape and normalize

def reshape_and_normalize(images):
    #Reshape the images to add an extra dimension
    images = images.reshape((-1, 28, 28, 1))

    # Normalize the pixel values
    images = images /255.0
    return images

In [4]:
# reloading the images in case you run this cell multiple times
(training_images, training_labels) , _  = tf.keras.datasets.mnist.load_data(path=data_path)

# applying the function
training_images = reshape_and_normalize(training_images)

print(f'Maximum pixel value after normalization: {np.max(training_images)}\n')
print(f'Shape of training set after reshaping: {training_images.shape}\n')
print(f'Shape of one image after reshaping: {training_images[0].shape}')

Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


## Defining Callback

In [19]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.995):
            print("\n Reached 99.5% accuracy so cancelling training!")
            self.model.stop_training = True

In [6]:
# convolutional_model
def convolutional_model():

    #Defining the model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [20]:
model = convolutional_model()

#Get number of weights
model_params = model.count_params()

assert model_params < 1000000, (
    f'Your model has {model_params:,}params. For successful graidng, please keep it'
    f'under 1,000,000 by reducing the number of units in your Conv2D and Dense layers.'
)

# Instantiate callback class
callbacks = myCallback()

# Training the model
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.1232 - accuracy: 0.9630
Epoch 2/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0400 - accuracy: 0.9880
Epoch 3/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0253 - accuracy: 0.9921
Epoch 4/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0181 - accuracy: 0.9943
Epoch 5/10
1874/1875 [============================>.] - ETA: 0s - loss: 0.0131 - accuracy: 0.9956
 Reached 99.5% accuracy so cancelling training!
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0131 - accuracy: 0.9956
